In [ ]:
import os
from IPython.core.debugger import set_trace
from models.Mini_REDCNN import REDCNN
from utils import utils
import torch
import torch.nn as nn
import torch.utils.data.sampler as sampler
from torch.autograd import Variable
from matplotlib import pyplot as plt
import random
from numpy import log10

In [ ]:
#dataset
csv_file = r'C:\Users\Juan Pisula\Desktop\ct_images\para probar el procesamiento de los volumenes\100_FBPPhil_500FBP.csv'
csv_file = r'C:\Users\Juan Pisula\Desktop\ct_images\patches\100_FBPPhil_500FBP.csv'
dataset = utils.CTVolumesDataset(csv_file)

#split of data in training data and validation data
num_samples = len(dataset)
training_samples_percentage = 0.8
split_index = int( num_samples * training_samples_percentage )
total_idx = list(range(num_samples))
random.seed(10)
random.shuffle(total_idx)
training_idx = total_idx[0 : split_index]
validation_idx = total_idx[split_index : num_samples]
training_sampler = sampler.SubsetRandomSampler(training_idx)
validation_sampler = sampler.SubsetRandomSampler(validation_idx)

#dataloader
batch_size = 16
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size = batch_size, sampler = validation_sampler)

In [ ]:
net_dir = r'C:\Users\Juan Pisula\Desktop\ct_images\3d_autoencoder_epoch_15.pkl'
model = REDCNN()
model.load_state_dict(torch.load(net_dir))
model.double()
if torch.cuda.is_available():
    model.cuda()
model.eval()

In [ ]:
#batches = list(dataloader)

In [ ]:
vol_idx = 50
slc = 10
pair = dataset[vol_idx]
lo_res = pair[0]
target = pair[1]

fig = plt.figure()
a = fig.add_subplot(1, 2, 1)
imgplot = plt.imshow(lo_res[slc][:][:])
a = fig.add_subplot(1, 2, 2)
imgplot = plt.imshow(target[slc][:][:])

In [ ]:
lo_res_ = torch.from_numpy(lo_res)
lo_res_ = lo_res_.unsqueeze(0)
lo_res_ = lo_res_.unsqueeze(1)
lo_res_ = utils.var_or_cuda( lo_res_ )
net_output = model(lo_res_)     

In [ ]:
fig = plt.figure()
a = fig.add_subplot(1, 3, 1)
imgplot = plt.imshow(lo_res[slc][:][:])
a = fig.add_subplot(1, 3, 2)
net_output_cpu = net_output.detach()
net_output_cpu = net_output_cpu.cpu()
imgplot = plt.imshow(net_output_cpu[0][0][slc][:][:])
a = fig.add_subplot(1, 3, 3)
imgplot = plt.imshow(target[slc][:][:])